In [127]:
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv
from pymongo import MongoClient
import json
import requests
import os

In [128]:
token = os.getenv("token")

In [129]:
client = MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("usa_companiess")
c.find_one()

{'_id': ObjectId('654624e51d17ce5e07c16dd7'),
 'name': 'hi5',
 'category_code': 'games_video',
 'number_of_employees': 100,
 'offices': [{'city': 'San Francisco',
   'country_code': 'USA',
   'latitude': 37.788668,
   'longitude': -122.400558}]}

In [130]:
path = "data/usa_companiess.json"
df = pd.read_json(path)
df

,name,category_code,number_of_employees,offices
0,hi5,games_video,100,"[{'city': 'San Francisco', 'country_code': 'US..."
1,Social Gaming Network,games_video,100,"[{'city': 'Los Angeles', 'country_code': 'USA'..."
2,VBS TV,games_video,100,"[{'city': 'Brooklyn', 'country_code': 'USA', '..."
3,ZillionTV,games_video,100,"[{'city': 'Sunnyvale', 'country_code': 'USA', ..."
4,Exent,games_video,100,"[{'city': 'New York', 'country_code': 'USA', '..."
5,iWin,games_video,103,"[{'city': 'San Francisco', 'country_code': 'US..."
6,RockYou,games_video,106,"[{'city': 'Redwood City', 'country_code': 'USA..."
7,G5 Entertainment,games_video,107,"[{'city': 'Kharkov', 'country_code': 'UKR', 'l..."
8,Zynga,games_video,115,"[{'city': 'San Francisco', 'country_code': 'US..."
9,Livestream,games_video,120,"[{'city': 'New York', 'country_code': 'USA', '..."


### 1st city: New York, NY

In [131]:
#Polygons??
# location = [40.761979, -74.026033]
# position = {"type": "Point", "coordinates":location}
# projection = {"name": 1, "_id": 0}
# c.find_one({"geometry":{"$geoIntersects":{"$geometry":position}}}, projection)["name"]
# Tried to do GeoJSON but I had problems with indexes in Mongo

In [132]:
lat = 40.761979
lon = -74.026033

def requests_for_foursquare(query, lat, lon):
        url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}"

        headers = {"accept": "application/json","Authorization": token}
        try:
            return requests.get(url, headers=headers).json()
        except Exception as e:
            print("Error:", e)
            return None
        
def requests_for_foursquare_2queries(query,query2, lat, lon):
        url = f"https://api.foursquare.com/v3/places/search?query={query1}%20%26%20{query2}&ll={lat}%2C{lon}"

        headers = {"accept": "application/json","Authorization": token}
        try:
            return requests.get(url, headers=headers).json()
        except Exception as e:
            print("Error:", e)
            return None
query = ["starbucks","schools","airport","club","vegan","basketball"]
query1= ["pet"]
query2 = ["grooming"]

In [133]:
def apply_request_starbucks():
    ny_starbucks = []
    for i in query: 
        if i == "starbucks":
            result = requests_for_foursquare("starbucks", lat, lon)
            ny_starbucks.append(result)
    return ny_starbucks

def apply_request_school():
    ny_schools = []
    for i in query:
        if i == "schools":
            result = requests_for_foursquare("schools", lat, lon)
            ny_schools.append(result)
    return ny_schools

def apply_request_airports():
    ny_airport = []
    for i in query:
        if i == "airport":
            result = requests_for_foursquare("airport", lat, lon)
            ny_airport.append(result)
    return ny_airport

def apply_request_clubs():
    ny_clubs = []
    for i in query:
        if i == "club":
            result = requests_for_foursquare("club", lat, lon)
            ny_clubs.append(result)
    return ny_clubs

def apply_request_vegan():
    ny_vegan = []
    for i in query:
        if i == "vegan":
            result = requests_for_foursquare("vegan", lat, lon)
            ny_vegan.append(result)
    return ny_vegan

def apply_request_basketball():
    ny_basketball = []
    for i in query:
        if i == "basketball":
            result = requests_for_foursquare("basketball", lat, lon)
            ny_basketball.append(result)
    return ny_basketball

def apply_request_pet():
    ny_pet = []
    result = requests_for_foursquare_2queries("pet","grooming",lat, lon)
    ny_pet.append(result)  
    return ny_pet

In [134]:
new_dict = {}

ny_starbucks = apply_request_starbucks()
starbucks_names = [result["name"] for i in ny_starbucks for result in i.get("results", [])]
new_dict["Starbucks_n"] = starbucks_names
starbucks_latitudes = [result["geocodes"]["main"]["latitude"] for i in ny_starbucks for result in i.get("results", [])]
new_dict["Starbucks_lat"] = starbucks_latitudes
starbucks_longitudes = [result["geocodes"]["main"]["longitude"] for i in ny_starbucks for result in i.get("results", [])]
new_dict["Starbucks_lon"] = starbucks_longitudes

ny_schools = apply_request_school()
schools_names = [result["name"] for i in ny_schools for result in i.get("results", [])]
new_dict["Schools_n"] = schools_names
schools_latitudes = [result["geocodes"]["main"]["latitude"] for i in ny_schools for result in i.get("results", [])]
new_dict["Schools_lat"] = schools_latitudes
schools_longitudes = [result["geocodes"]["main"]["longitude"] for i in ny_schools for result in i.get("results", [])]
new_dict["Schools_lon"] = schools_longitudes

ny_airport = apply_request_airports()
airport_name = [result["name"] for i in ny_airport for result in i.get("results", [])]
new_dict["Airport_n"] = airport_name
airport_latitude = [result["geocodes"]["main"]["latitude"] for i in ny_airport for result in i.get("results", [])]
new_dict["Airport_lat"] = airport_latitude
airport_longitude = [result["geocodes"]["main"]["longitude"] for i in ny_airport for result in i.get("results", [])]
new_dict["Airport_lon"] = airport_longitude

ny_clubs = apply_request_clubs()
clubs_name = [result["name"] for i in ny_clubs for result in i.get("results", [])]
new_dict["Clubs_n"] = clubs_name
club_latitude = [result["geocodes"]["main"]["latitude"] for i in ny_clubs for result in i.get("results", [])]
new_dict["Clubs_lat"] = club_latitude
club_longitude = [result["geocodes"]["main"]["longitude"] for i in ny_clubs for result in i.get("results",[])]
new_dict["Clubs_lon"] = club_longitude

ny_vegan = apply_request_vegan()
vegan_name = [result["name"] for i in ny_vegan for result in i.get("results", [])]
new_dict["Vegan_n"] = vegan_name
vegan_latitude = [result["geocodes"]["main"]["latitude"] for i in ny_vegan for result in i.get("results", [])]
new_dict["Vegan_lat"] = vegan_latitude
vegan_longitude = [result["geocodes"]["main"]["longitude"] for i in ny_vegan for result in i.get("result", [])]
new_dict["Vegan_lon"] = vegan_latitude

ny_basketball = apply_request_basketball()
basketball_name = [result["name"] for i in ny_basketball for result in i.get("results",[])]
new_dict["Basketball_n"] = basketball_name
basketball_latitude = [result["geocodes"]["main"]["latitude"] for i in ny_basketball for result in i.get("results", [])]
new_dict["Basketball_lat"] = basketball_latitude
basketball_longitde = [result["geocodes"]["main"]["longitude"] for i in ny_basketball for result in i.get("results", [])]
new_dict["Basketball_lon"] = basketball_longitde

ny_pet = apply_request_pet()
pet_name = [result["name"] for i in ny_pet for result in i.get("results",[])]
new_dict["Pet_n"] = pet_name
pet_latitude = [result["geocodes"]["main"]["latitude"] for i in ny_pet for result in i.get("results", [])]
new_dict["Pet_lat"] = pet_latitude
pet_longitde = [result["geocodes"]["main"]["longitude"] for i in ny_pet for result in i.get("results", [])]
new_dict["Pet_lon"] = pet_longitde


In [135]:
ny_companies = pd.DataFrame(new_dict)
ny_companies

,Starbucks_n,Starbucks_lat,Starbucks_lon,Schools_n,Schools_lat,Schools_lon,Airport_n,Airport_lat,Airport_lon,Clubs_n,...,Clubs_lon,Vegan_n,Vegan_lat,Vegan_lon,Basketball_n,Basketball_lat,Basketball_lon,Pet_n,Pet_lat,Pet_lon
0,Starbucks,40.751167,-74.025307,Hopscotch Montessori School - Via 57,40.771419,-73.993038,Teterboro Airport (TEB),40.854269,-74.061662,Classic Car Club Manhattan,...,-74.004615,Red Bamboo,40.731292,40.731292,Basketball Court,40.732496,-73.979550,Pet Star Grooming,40.759426,-73.927081
1,Starbucks,40.741623,-74.005316,Lrei,40.729362,-74.001906,Airport,40.752971,-73.986765,Gotham Comedy Club,...,-73.996419,Franchia,40.747505,40.747505,Riverside Park Basketball Courts,40.782714,-73.984522,A1 Pet Grooming,40.774894,-73.956204
2,Starbucks,40.783022,-74.007045,Robert Fulton School,40.797260,-74.005554,The Centurion Lounge,40.774530,-73.871854,New York Athletic Club,...,-73.978821,Peacefood Cafe,40.785397,40.785397,Basketball City,40.709975,-73.984905,Hoboken Pet,40.743016,-74.029392
3,Starbucks,40.774082,-74.037894,William T Sherman School,40.781918,-73.978022,Meridian Teterboro,40.850760,-74.069355,Sam's Club,...,-74.043673,Vinnie's Pizzeria,40.718927,40.718927,Rucker Park Basketball Courts,40.829789,-73.936404,Jersey City Unleashed Pet Resort and Spa,40.732695,-74.039807
4,Starbucks,40.740618,-74.030256,Rodeph Sholom Day School,40.784098,-73.971245,United Polaris Lounge,40.696572,-74.177370,Birdland,...,-73.989624,Modern Love Brooklyn,40.709803,40.709803,Hudson River Park Basketball Court,40.718816,-74.012838,Cozy Cuts Pet Grooming,40.743718,-74.037238
5,Starbucks,40.749124,-73.991375,Manhattan School For Children,40.791503,-73.970877,Delta SkyClub & SkyDeck,40.638934,-73.781225,Soho House US,...,-74.005930,Le Botaniste,40.720348,40.720348,14th Street Viaduct Basketball Court,40.754543,-74.034048,Barba Men's Grooming Boutique,40.742340,-73.998794
6,Starbucks,40.761533,-73.997411,Marymount School/Pratt Mansions,40.779884,-73.961335,Virgin Atlantic Clubhouse Lounge,40.643287,-73.779546,Mezzrow,...,-74.001853,The Butcher's Daughter,40.720658,40.720658,Basketball Gym At The Dylan,40.746752,-73.985298,Pet Central,40.738710,-73.995376
7,Starbucks,40.752765,-73.998376,Little Dalton - First Program,40.783920,-73.955722,Emirates First & Business Class Lounge,40.643091,-73.778755,Le Bain,...,-74.008170,Paulie Gee's,40.729615,40.729615,NY Urban Basketball,40.785911,-73.974069,Pet Central,40.744922,-73.997105
8,Starbucks,40.740445,-74.005008,Public School 307,40.701757,-73.981703,Essex County Airport (CDW),40.875820,-74.277527,Five Iron Golf,...,-73.991811,Dirt Candy,40.717855,40.717855,Tompkins Square Park Basketball Courts,40.726974,-73.980550,Towne House Grooming,40.746203,-73.997593
9,Starbucks,40.769232,-73.988102,P.S. 29 Auditorium,40.686926,-73.997743,American Airlines Flagship Lounge,40.648120,-73.791671,Ludlow House,...,-73.988417,Uptown Veg,40.805436,40.805436,Basketball City - Trevor Day,40.786967,-73.968909,Doggie Styles Pet Grooming,40.711202,-73.959379


In [136]:
lat = 40.761979
lon = -74.026033
ny_map = folium.Map(location = [lat, lon], zoom_start = 15)


In [137]:
def add_marker(map, lat, lon, icon_name, color, icon_color):
    icon = Icon(
        icon=icon_name,
        prefix="fa",
        color=color,
        icon_color=icon_color
    )
    new_marker = folium.Marker(location=[lat, lon], icon=icon)
    new_marker.add_to(map)

for index, row in ny_companies.iterrows():
    if row["Starbucks_n"] == "Starbucks":
        add_marker(ny_map, row["Starbucks_lat"], row["Starbucks_lon"], "coffee", "white", "green")

    if row["Schools_n"]:
        add_marker(ny_map, row["Schools_lat"], row["Schools_lon"], "graduation-cap", "blue", "white")

    if row["Airport_n"]:
        add_marker(ny_map, row["Airport_lat"], row["Airport_lon"], "plane", "gray", "white")

    if row["Clubs_n"]:
        add_marker(ny_map, row["Clubs_lat"], row["Clubs_lon"], "beer", "lightred", "white")

    if row["Vegan_n"]:
        add_marker(ny_map, row["Vegan_lat"], row["Vegan_lon"], "leaf", "darkgreen", "white")

    if row["Basketball_n"]:
        add_marker(ny_map, row["Basketball_lat"], row["Basketball_lon"], "futbol", "orange", "black")

    if row["Pet_n"]:
        add_marker(ny_map, row["Pet_lat"], row["Pet_lon"], "paw", "purple", "white")

ny_map


### 2nd city: San Francisco, CA

In [138]:
lat = 37.788668
lon = -122.400558

def apply_request_starbucks():
    sf_starbucks = []
    for i in query: 
        if i == "starbucks":
            result = requests_for_foursquare("starbucks", lat, lon)
            sf_starbucks.append(result)
    return sf_starbucks

def apply_request_school():
    sf_schools = []
    for i in query:
        if i == "schools":
            result = requests_for_foursquare("schools", lat, lon)
            sf_schools.append(result)
    return sf_schools

def apply_request_airports():
    sf_airport = []
    for i in query:
        if i == "airport":
            result = requests_for_foursquare("airport", lat, lon)
            sf_airport.append(result)
    return sf_airport

def apply_request_clubs():
    sf_clubs = []
    for i in query:
        if i == "club":
            result = requests_for_foursquare("club", lat, lon)
            sf_clubs.append(result)
    return sf_clubs

def apply_request_vegan():
    sf_vegan = []
    for i in query:
        if i == "vegan":
            result = requests_for_foursquare("vegan", lat, lon)
            sf_vegan.append(result)
    return sf_vegan

def apply_request_basketball():
    sf_basketball = []
    for i in query:
        if i == "basketball":
            result = requests_for_foursquare("basketball", lat, lon)
            sf_basketball.append(result)
    return sf_basketball

def apply_request_pet():
    sf_pet = []
    result = requests_for_foursquare_2queries("pet","grooming",lat, lon)
    sf_pet.append(result)  
    return sf_pet

In [139]:
sf_dict = {}

sf_starbucks = apply_request_starbucks()
starbucks_names = [result["name"] for i in sf_starbucks for result in i.get("results", [])]
sf_dict["Starbucks_n"] = starbucks_names
starbucks_latitudes = [result["geocodes"]["main"]["latitude"] for i in sf_starbucks for result in i.get("results", [])]
sf_dict["Starbucks_lat"] = starbucks_latitudes
starbucks_longitudes = [result["geocodes"]["main"]["longitude"] for i in sf_starbucks for result in i.get("results", [])]
sf_dict["Starbucks_lon"] = starbucks_longitudes

sf_schools = apply_request_school()
schools_names = [result["name"] for i in sf_schools for result in i.get("results", [])]
sf_dict["Schools_n"] = schools_names
schools_latitudes = [result["geocodes"]["main"]["latitude"] for i in sf_schools for result in i.get("results", [])]
sf_dict["Schools_lat"] = schools_latitudes
schools_longitudes = [result["geocodes"]["main"]["longitude"] for i in sf_schools for result in i.get("results", [])]
sf_dict["Schools_lon"] = schools_longitudes

sf_airport = apply_request_airports()
airport_name = [result["name"] for i in sf_airport for result in i.get("results", [])]
sf_dict["Airport_n"] = airport_name
airport_latitude = [result["geocodes"]["main"]["latitude"] for i in sf_airport for result in i.get("results", [])]
sf_dict["Airport_lat"] = airport_latitude
airport_longitude = [result["geocodes"]["main"]["longitude"] for i in sf_airport for result in i.get("results", [])]
sf_dict["Airport_lon"] = airport_longitude

sf_clubs = apply_request_clubs()
clubs_name = [result["name"] for i in sf_clubs for result in i.get("results", [])]
sf_dict["Clubs_n"] = clubs_name
club_latitude = [result["geocodes"]["main"]["latitude"] for i in sf_clubs for result in i.get("results", [])]
sf_dict["Clubs_lat"] = club_latitude
club_longitude = [result["geocodes"]["main"]["longitude"] for i in sf_clubs for result in i.get("results",[])]
sf_dict["Clubs_lon"] = club_longitude

sf_vegan = apply_request_vegan()
vegan_name = [result["name"] for i in sf_vegan for result in i.get("results", [])]
sf_dict["Vegan_n"] = vegan_name
vegan_latitude = [result["geocodes"]["main"]["latitude"] for i in sf_vegan for result in i.get("results", [])]
sf_dict["Vegan_lat"] = vegan_latitude
vegan_longitude = [result["geocodes"]["main"]["longitude"] for i in sf_vegan for result in i.get("result", [])]
sf_dict["Vegan_lon"] = vegan_latitude

sf_basketball = apply_request_basketball()
basketball_name = [result["name"] for i in sf_basketball for result in i.get("results",[])]
sf_dict["Basketball_n"] = basketball_name
basketball_latitude = [result["geocodes"]["main"]["latitude"] for i in sf_basketball for result in i.get("results", [])]
sf_dict["Basketball_lat"] = basketball_latitude
basketball_longitde = [result["geocodes"]["main"]["longitude"] for i in sf_basketball for result in i.get("results", [])]
sf_dict["Basketball_lon"] = basketball_longitde

sf_pet = apply_request_pet()
pet_name = [result["name"] for i in sf_pet for result in i.get("results",[])]
sf_dict["Pet_n"] = pet_name
pet_latitude = [result["geocodes"]["main"]["latitude"] for i in sf_pet for result in i.get("results", [])]
sf_dict["Pet_lat"] = pet_latitude
pet_longitde = [result["geocodes"]["main"]["longitude"] for i in sf_pet for result in i.get("results", [])]
sf_dict["Pet_lon"] = pet_longitde

sf_companies = pd.DataFrame(sf_dict)
sf_companies

,Starbucks_n,Starbucks_lat,Starbucks_lon,Schools_n,Schools_lat,Schools_lon,Airport_n,Airport_lat,Airport_lon,Clubs_n,...,Clubs_lon,Vegan_n,Vegan_lat,Vegan_lon,Basketball_n,Basketball_lat,Basketball_lon,Pet_n,Pet_lat,Pet_lon
0,Starbucks,37.788398,-122.393628,UCSF Mission Bay,37.768855,-122.395848,San Francisco International Airport (SFO),37.616490,-122.386810,SFJAZZ Center,...,-122.421445,Shizen,37.768028,37.768028,We Basketball,37.875969,-122.271589,Village Pets & Supplies,37.790433,-122.417048
1,Starbucks,37.793812,-122.396743,La Petite Baleen Swim Schools,37.805384,-122.469116,The Centurion Lounge,37.617510,-122.387817,The Battery,...,-122.401217,Gracias Madre,37.761556,37.761556,UPBasketball - Unlimited Potential Basketball,37.598357,-122.382401,Pet Food Express,37.769316,-122.426193
2,Starbucks,37.794601,-122.402669,Sunset Elementary School,37.750938,-122.499613,United Polaris Lounge,37.617054,-122.389508,Great American Music Hall,...,-122.418866,Millennium,37.847471,37.847471,Berry Basketball Courts,37.772080,-122.397686,Bernie's Grooming City Dogs,37.783786,-122.389414
3,Starbucks,37.789098,-122.408438,Anderson's Swim School,37.648152,-122.490253,Emirates Lounge,37.613092,-122.389133,The Independent,...,-122.437670,Judahlicious,37.760656,37.760656,Alice Marble Basketball Court,37.801449,-122.420352,Jeffrey's Natural Pet Foods,37.801681,-122.411332
4,Starbucks,37.783998,-122.407206,Presidio Knolls School,37.773203,-122.413809,Delta Sky Club,37.615272,-122.383678,The Great Northern,...,-122.406625,Greens Restaurant,37.773837,37.773837,Hamilton Basketball Courts,37.785284,-122.435668,Wags Pet Wash & Boutique,37.793688,-122.421040
5,Starbucks,37.789293,-122.406729,Live Oak School,37.763509,-122.399117,Hayward Executive Airport (HWD),37.662811,-122.121239,Punch Line Comedy Club,...,-122.400068,Wildseed,37.797611,37.797611,Dolores Park Basketball Courts,37.761189,-122.427791,Pet Food Express,37.787635,-122.446343
6,Starbucks,37.786145,-122.405612,SF Public Montessori School Elementary School,37.792830,-122.433534,Alaska Lounge,37.617405,-122.382729,Rickshaw Stop,...,-122.420500,The Butcher's Son,37.871883,37.871883,Golden Gate Park Panhandle Basketball Courts,37.771890,-122.445911,The Refinery Grooming Club,37.755750,-122.418583
7,Starbucks,37.789534,-122.401869,Daniel Webster Elementary SCHL,37.760587,-122.395863,The Dining Room,37.616875,-122.389713,Cat Club,...,-122.409920,Enjoy Vegetarian Restaurant,37.795798,37.795798,Nate Thurmond Basketball Court,37.772186,-122.448436,Maddie's Pet Adoption Center,37.766210,-122.412106
8,Starbucks,37.786805,-122.407228,Tule Elk Park EES Family Council,37.799316,-122.434564,Virgin Atlantic Upper Class Clubhouse,37.614275,-122.388200,Black Cat,...,-122.414508,Souley Vegan,37.797037,37.797037,Chase Center Stadium,37.767924,-122.387478,Jeffrey's Natural Pet Foods,37.764520,-122.433363
9,Starbucks,37.808488,-122.412881,Creative Arts Charter School,37.779701,-122.436054,Cathay Pacific First & Business Class Lounge,37.614118,-122.388978,Madrone Art Bar,...,-122.437450,Sweetgreen,37.787275,37.787275,Bay City Basketball Headquarters,37.781075,-122.468385,Wolf & Lion Pet Supplies,37.799333,-122.442020


In [140]:
lat = 37.788668
lon = -122.400558
sf_map = folium.Map(location = [lat, lon], zoom_start = 15)

In [141]:
for index, row in sf_companies.iterrows():
    if row["Starbucks_n"] == "Starbucks":
        add_marker(sf_map, row["Starbucks_lat"], row["Starbucks_lon"], "coffee", "white", "green")

    if row["Schools_n"]:
        add_marker(sf_map, row["Schools_lat"], row["Schools_lon"], "graduation-cap", "blue", "white")

    if row["Airport_n"]:
        add_marker(sf_map, row["Airport_lat"], row["Airport_lon"], "plane", "gray", "white")

    if row["Clubs_n"]:
        add_marker(sf_map, row["Clubs_lat"], row["Clubs_lon"], "beer", "lightred", "white")

    if row["Vegan_n"]:
        add_marker(sf_map, row["Vegan_lat"], row["Vegan_lon"], "leaf", "darkgreen", "white")

    if row["Basketball_n"]:
        add_marker(sf_map, row["Basketball_lat"], row["Basketball_lon"], "futbol", "orange", "black")

    if row["Pet_n"]:
        add_marker(sf_map, row["Pet_lat"], row["Pet_lon"], "paw", "purple", "white")

sf_map

### 3rd city. Los Angeles, CA

In [152]:
lat = 34.085749
lon = -118.343791

def apply_request_starbucks():
    la_starbucks = []
    for i in query: 
        if i == "starbucks":
            result = requests_for_foursquare("starbucks", lat, lon)
            la_starbucks.append(result)
    return la_starbucks

def apply_request_school():
    la_schools = []
    for i in query:
        if i == "schools":
            result = requests_for_foursquare("schools", lat, lon)
            la_schools.append(result)
    return la_schools

def apply_request_airports():
    la_airport = []
    for i in query:
        if i == "airport":
            result = requests_for_foursquare("airport", lat, lon)
            la_airport.append(result)
    return la_airport

def apply_request_clubs():
    la_clubs = []
    for i in query:
        if i == "club":
            result = requests_for_foursquare("club", lat, lon)
            la_clubs.append(result)
    return la_clubs

def apply_request_vegan():
    la_vegan = []
    for i in query:
        if i == "vegan":
            result = requests_for_foursquare("vegan", lat, lon)
            la_vegan.append(result)
    return la_vegan

def apply_request_basketball():
    la_basketball = []
    for i in query:
        if i == "basketball":
            result = requests_for_foursquare("basketball", lat, lon)
            la_basketball.append(result)
    return la_basketball

def apply_request_pet():
    la_pet = []
    result = requests_for_foursquare_2queries("pet","grooming",lat, lon)
    la_pet.append(result)  
    return la_pet

In [154]:
la_dict = {}

la_starbucks = apply_request_starbucks()
starbucks_names = [result["name"] for i in la_starbucks for result in i.get("results", [])]
la_dict["Starbucks_n"] = starbucks_names
starbucks_latitudes = [result["geocodes"]["main"]["latitude"] for i in la_starbucks for result in i.get("results", [])]
la_dict["Starbucks_lat"] = starbucks_latitudes
starbucks_longitudes = [result["geocodes"]["main"]["longitude"] for i in la_starbucks for result in i.get("results", [])]
la_dict["Starbucks_lon"] = starbucks_longitudes

la_schools = apply_request_school()
schools_names = [result["name"] for i in la_schools for result in i.get("results", [])]
la_dict["Schools_n"] = schools_names
schools_latitudes = [result["geocodes"]["main"]["latitude"] for i in la_schools for result in i.get("results", [])]
la_dict["Schools_lat"] = schools_latitudes
schools_longitudes = [result["geocodes"]["main"]["longitude"] for i in la_schools for result in i.get("results", [])]
la_dict["Schools_lon"] = schools_longitudes

la_airport = apply_request_airports()
airport_name = [result["name"] for i in la_airport for result in i.get("results", [])]
la_dict["Airport_n"] = airport_name
airport_latitude = [result["geocodes"]["main"]["latitude"] for i in la_airport for result in i.get("results", [])]
la_dict["Airport_lat"] = airport_latitude
airport_longitude = [result["geocodes"]["main"]["longitude"] for i in la_airport for result in i.get("results", [])]
la_dict["Airport_lon"] = airport_longitude

la_clubs = apply_request_clubs()
clubs_name = [result["name"] for i in la_clubs for result in i.get("results", [])]
la_dict["Clubs_n"] = clubs_name
club_latitude = [result["geocodes"]["main"]["latitude"] for i in la_clubs for result in i.get("results", [])]
la_dict["Clubs_lat"] = club_latitude
club_longitude = [result["geocodes"]["main"]["longitude"] for i in la_clubs for result in i.get("results",[])]
la_dict["Clubs_lon"] = club_longitude

la_vegan = apply_request_vegan()
vegan_name = [result["name"] for i in la_vegan for result in i.get("results", [])]
la_dict["Vegan_n"] = vegan_name
vegan_latitude = [result["geocodes"]["main"]["latitude"] for i in la_vegan for result in i.get("results", [])]
la_dict["Vegan_lat"] = vegan_latitude
vegan_longitude = [result["geocodes"]["main"]["longitude"] for i in la_vegan for result in i.get("result", [])]
la_dict["Vegan_lon"] = vegan_latitude

la_basketball = apply_request_basketball()
basketball_name = [result["name"] for i in la_basketball for result in i.get("results",[])]
la_dict["Basketball_n"] = basketball_name
basketball_latitude = [result["geocodes"]["main"]["latitude"] for i in la_basketball for result in i.get("results", [])]
la_dict["Basketball_lat"] = basketball_latitude
basketball_longitde = [result["geocodes"]["main"]["longitude"] for i in la_basketball for result in i.get("results", [])]
la_dict["Basketball_lon"] = basketball_longitde

la_pet = apply_request_pet()
pet_name = [result["name"] for i in la_pet for result in i.get("results",[])]
la_dict["Pet_n"] = pet_name
pet_latitude = [result["geocodes"]["main"]["latitude"] for i in la_pet for result in i.get("results", [])]
la_dict["Pet_lat"] = pet_latitude
pet_longitde = [result["geocodes"]["main"]["longitude"] for i in la_pet for result in i.get("results", [])]
la_dict["Pet_lon"] = pet_longitde

la_companies = pd.DataFrame(la_dict)
la_companies

,Starbucks_n,Starbucks_lat,Starbucks_lon,Schools_n,Schools_lat,Schools_lon,Airport_n,Airport_lat,Airport_lon,Clubs_n,...,Clubs_lon,Vegan_n,Vegan_lat,Vegan_lon,Basketball_n,Basketball_lat,Basketball_lon,Pet_n,Pet_lat,Pet_lon
0,Starbucks,34.098146,-118.343489,UCLA Stein Eye Institute,34.064961,-118.443901,Los Angeles International Airport (LAX),33.943520,-118.408420,Academy Night Club,...,-118.320849,Sage Organic Vegan Bistro,34.077099,34.077099,Venice Beach Basketball Courts,33.986028,-118.472826,The Urban Pet,34.076414,-118.353962
1,Starbucks,34.075883,-118.344968,Center For Early Education,34.081609,-118.375567,Santa Monica Airport (SMO),34.016640,-118.447479,Club Monaco,...,-118.377079,Crossroads,34.083455,34.083455,West Hollywood Park Basketball Courts,34.082923,-118.384364,Hammer & Nails - Hand & Foot Grooming Shop,34.083890,-118.369316
2,Starbucks,34.083539,-118.356152,Horace Mann School,34.065357,-118.382038,Van Nuys Airport (VNY),34.201248,-118.492020,Hollywood Improv,...,-118.367471,Cafe Gratitude,34.082769,34.082769,Tinder Basketball Court,34.091004,-118.384310,Laurel Pet Hospital,34.090601,-118.364000
3,Starbucks,34.090394,-118.326288,Pio Pico Middle School,34.045367,-118.317150,Whiteman Airport (WHP),34.254841,-118.408653,Laugh Factory,...,-118.364504,Hugo's Restaurant,34.090824,34.090824,STAPLES Center,34.043102,-118.267060,Kings & Queens Unisex Grooming Bar,34.083725,-118.346959
4,Starbucks,34.070048,-118.343582,Beverly Vista Student Council,34.062004,-118.394701,Hollywood Burbank Airport (BUR),34.196434,-118.354230,The Highlight Room,...,-118.330153,SunCafe Organic,34.138542,34.138542,Silver Lake Basketball Court,34.092115,-118.266032,Hammer & Nails Grooming Shop for Guys,34.090812,-118.345319
5,Starbucks,34.098717,-118.348452,Parking Entrance for UCLA Community School,34.058007,-118.297726,Burbank Airport Ride Share Pick Up,34.195776,-118.354300,Hollywood Palladium,...,-118.324249,Cafe Gratitude,33.997949,33.997949,Mulholland Tennis Club,34.118363,-118.390651,The Urban Pet,34.091353,-118.343428
6,Starbucks,34.101741,-118.337331,Hawthorne Elementary School,34.079055,-118.403949,American Airlines Flagship Lounge,33.941379,-118.406870,Hudson Theatres,...,-118.332241,Gracias Madre West Hollywood,34.080973,34.080973,Bell-Jeff Basketball Gym,34.184162,-118.305638,D Pet Hotels,34.090055,-118.338946
7,Starbucks,34.061893,-118.338898,Loyola High School,34.045025,-118.295641,Qantas First Class Lounge,33.943089,-118.409235,The Comedy Store,...,-118.373994,Veggie Grill,34.073605,34.073605,Staples Center Luxury Suites,34.043123,-118.267239,Cahuenga Pet Hospital,34.086137,-118.329107
8,Starbucks,34.094558,-118.309527,Alexander Hamilton High School,34.034148,-118.391676,Star Alliance Lounge,33.943897,-118.410311,Sound Nightclub,...,-118.335832,Hugo's Studio City,34.157846,34.157846,Basketball Courts,34.176615,-118.304007,Furtographs Pet Portraiture,34.076414,-118.353962
9,Starbucks,34.097423,-118.322460,Linwood E Howe Elementary School,34.021651,-118.392207,Delta Sky Club,33.946243,-118.404212,The Teragram Ballroom,...,-118.267117,Stuff I Eat,33.963328,33.963328,Champions,33.964820,-118.353636,L. A. Pet Clinic,34.083640,-118.327998


In [156]:
lat = 34.085749
lon =  -118.343791
la_map = folium.Map(location = [lat, lon], zoom_start = 15)

In [157]:
for index, row in la_companies.iterrows():
    if row["Starbucks_n"] == "Starbucks":
        add_marker(la_map, row["Starbucks_lat"], row["Starbucks_lon"], "coffee", "white", "green")

    if row["Schools_n"]:
        add_marker(la_map, row["Schools_lat"], row["Schools_lon"], "graduation-cap", "blue", "white")

    if row["Airport_n"]:
        add_marker(la_map, row["Airport_lat"], row["Airport_lon"], "plane", "gray", "white")

    if row["Clubs_n"]:
        add_marker(la_map, row["Clubs_lat"], row["Clubs_lon"], "beer", "lightred", "white")

    if row["Vegan_n"]:
        add_marker(la_map, row["Vegan_lat"], row["Vegan_lon"], "leaf", "darkgreen", "white")

    if row["Basketball_n"]:
        add_marker(la_map, row["Basketball_lat"], row["Basketball_lon"], "futbol", "orange", "black")

    if row["Pet_n"]:
        add_marker(la_map, row["Pet_lat"], row["Pet_lon"], "paw", "purple", "white")

la_map